In [2]:
import pandas as pd
import os
import torch
from datasets import load_dataset
import textwrap

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, T5Tokenizer, MBartTokenizer, MBartForConditionalGeneration, MT5Tokenizer, MT5ForConditionalGeneration


# Определение лучшего чекпоинта для каждой модели

In [14]:
metrics_ruT5 = pd.read_csv('metrics/metrics_ruT5.csv')
metrics_ruT5

,checkpoint,gen_len,rouge1_f,rouge2_f,rougel_f,bert_score_f1,chrf++,bleu,meteor
0,checkpoint-1,28.5254,27.5093,12.8689,24.9481,76.9721,36.8263,8.7239,25.7877
1,checkpoint-5,29.5675,29.6157,14.3995,26.8771,77.9207,39.0962,10.2914,28.3542
2,checkpoint-10,29.5421,30.4751,15.0693,27.7050,78.2533,39.8055,10.7023,29.0795
3,checkpoint-15,29.6419,30.6070,15.1734,27.8276,78.3665,39.9895,10.7479,29.2643
4,checkpoint-20,29.7094,30.7346,15.2225,27.9426,78.3572,40.0573,10.9116,29.4225


Последний чекпоинт для модели ruT5-base оказался наилучшим по всем метрикам

In [15]:
metrics_rugpt3small = pd.read_csv('metrics/metrics_rugpt3small.csv')
metrics_rugpt3small

,checkpoint,gen_len,rouge1_f,rouge2_f,rougel_f,bert_score_f1,chrf++,bleu,meteor
0,checkpoint-1,24.9628,22.3090,9.3042,20.1036,73.3188,29.3804,6.0272,21.5824
1,checkpoint-10,22.3563,24.6596,11.2175,22.3052,74.7099,32.5906,8.6051,24.6576
2,checkpoint-20,19.0366,25.4454,11.6954,23.0045,75.4493,32.9665,9.3725,25.2293
3,checkpoint-30,19.3306,25.7735,11.9392,23.3381,75.5763,33.2885,9.5952,25.5423
4,checkpoint-40,18.2206,25.7682,11.9178,23.3153,75.6450,33.3899,9.5557,25.6184


Последний чекпоинт для модели ruGPT3-small оказался наилучшим почти по всем метрикам

In [ ]:
metrics_rugpt3medium1 = pd.read_csv('metrics/metrics_rugpt3medium1.csv')
metrics_rugpt3medium2 = pd.read_csv('metrics/metrics_rugpt3medium2.csv')
metrics_rugpt3medium = pd.concat([metrics_rugpt3medium1, metrics_rugpt3medium2], ignore_index=True)
metrics_rugpt3medium

,checkpoint,gen_len,rouge1_f,rouge2_f,rougel_f,bert_score_f1,chrf++,bleu,meteor
0,checkpoint-1,16.8448,27.6208,13.1655,24.7906,76.6617,34.5505,10.2944,27.2294
1,checkpoint-2,17.0465,27.5998,13.1305,24.8251,76.8588,35.2836,10.3490,27.5693
2,checkpoint-3,17.3789,28.0156,13.5553,25.2542,76.9728,35.6485,10.6785,27.8397
3,checkpoint-5,17.3086,28.3472,13.6715,25.6117,77.1060,36.0550,10.9312,28.2693
4,checkpoint-10,17.9393,28.5318,13.7822,25.7967,77.0877,37.1313,11.3196,28.8166
5,checkpoint-20,23.2076,27.0257,12.5114,24.4839,76.1750,38.8347,10.7231,29.0458


Для модели ruGPT3-medium наилучшим по большинству метрикам оказался 20 чекпоинт

## Вывод

Для оценки предиктов будем использовать следующие чекпоинты для моделей

- ruT5-base - checkpoint-20
- ruGPT3-small-base - checkpoint-40
- ruGPT3-medium - checkpoint-20

# Оценка предиктов моделей

In [ ]:

WRAP_WIDTH = 160

def init_gpt(tokenizer_name, model_path):
    tok = AutoTokenizer.from_pretrained(tokenizer_name)
    tok.pad_token = tok.eos_token
    mdl = AutoModelForCausalLM.from_pretrained(model_path)
    mdl.config.pad_token_id = tok.pad_token_id
    mdl.to(device).eval()
    return tok, mdl


def init_seq2seq(tok_class, tokenizer_name, model_path):
    tok = tok_class.from_pretrained(tokenizer_name)
    mdl = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    mdl.to(device).eval()
    return tok, mdl


MODELS = {
    "ruGPT3medium": {"type": "gpt", "tokenizer": (AutoTokenizer, "sberbank-ai/rugpt3medium_based_on_gpt2"), "model": (None, "checkpoints/ruGPT3medium_sum/checkpoint-20"), "max_length": 64},
    "ruGPT3small" : {"type": "gpt", "tokenizer": (AutoTokenizer, "sberbank-ai/rugpt3small_based_on_gpt2"), "model": (None, "checkpoints/ruGPT3small_sum/checkpoint-40"), "max_length": 64},
    "ruT5"        : {"type": "t5",    "tokenizer": (T5Tokenizer, "ai-forever/ruT5-base"),                   "model": (None, "checkpoints/ruT5_sum/checkpoint-20"),           "max_length": 64, "gen_kwargs": {"num_beams": 4, "no_repeat_ngram_size": 4, "early_stopping": True}},
    "mbart"       : {"type": "mbart", "tokenizer": (MBartTokenizer, "IlyaGusev/mbart_ru_sum_gazeta"), "model": (MBartForConditionalGeneration, "IlyaGusev/mbart_ru_sum_gazeta"), "max_length": 200, "gen_kwargs": {"num_beams": 5, "no_repeat_ngram_size": 4, "early_stopping": True}},
    "mt5"         : {"type": "mt5",   "tokenizer": (MT5Tokenizer, "csebuetnlp/mT5_multilingual_XLSum"),    "model": (MT5ForConditionalGeneration, "csebuetnlp/mT5_multilingual_XLSum"), "max_length": 84, "gen_kwargs": {"num_beams": 4, "no_repeat_ngram_size": 2, "early_stopping": True}},}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def summarize_text(user_text: str) -> dict:
    results = {}

    for name, cfg in MODELS.items():
        # инициализация
        tok_class, tok_name = cfg['tokenizer']
        mdl_class, mdl_path = cfg['model']
        if cfg['type'] == 'gpt':
            tokenizer, model = init_gpt(tok_name, mdl_path)
        elif cfg['type'] == 't5':
            tokenizer, model = init_seq2seq(T5Tokenizer, tok_name, mdl_path)
        elif cfg['type'] == 'mbart':
            tokenizer = MBartTokenizer.from_pretrained(tok_name)
            tokenizer.src_lang = 'ru_RU'
            model = MBartForConditionalGeneration.from_pretrained(mdl_path).to(device).eval()
        elif cfg['type'] == 'mt5':
            tokenizer, model = init_seq2seq(MT5Tokenizer, tok_name, mdl_path)


        if cfg['type'] == 'gpt':
            prompt = f"Сделай краткое содержание следующего текста и только не придумывай то, чего нет в тексте: {user_text} Сводка:"
            inputs = tokenizer(prompt, return_tensors='pt', truncation=True).to(device)
            gen_ids = model.generate(
                inputs.input_ids,
                max_new_tokens=cfg['max_length'],
                do_sample=True,
                top_p=0.9,
                no_repeat_ngram_size=2,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )
            summary = tokenizer.decode(gen_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
        else:
            inputs = tokenizer(user_text, return_tensors='pt', truncation=True, padding='longest', max_length=512).to(device)
            gen_kwargs = cfg.get('gen_kwargs', {})
            if cfg['type'] == 'mbart':
                gen_kwargs['forced_bos_token_id'] = tokenizer.lang_code_to_id[tokenizer.src_lang]
            gen_ids = model.generate(**inputs,max_length=cfg['max_length'],**gen_kwargs)
            summary = tokenizer.decode(gen_ids[0], skip_special_tokens=True).strip()
            
        results[name] = summary
    return results




In [28]:
text = input()
print('Новость')
print(textwrap.fill(text, width=WRAP_WIDTH))
summaries = summarize_text(text)
for model_name, summary in summaries.items():
    print(f"\n--- {model_name} summary ---")
    print(textwrap.fill(summary, width=WRAP_WIDTH))

Новость
Число погибших от торнадо в США увеличилось до 27 Как минимум 27 человек погибли в результате торнадо и штормов в центральной части США, передает The New York
Times. Из них 25 находились в штатах Миссури и Кентукки, 2 — в Северной Вирджинии.  В Кентукки еще не менее десяти человек получили ранения, их состояние
оценивается как тяжелое, сообщил губернатор Энди Бешир. В штате Висконсин в результате торнадо без света остались сотни тысяч человек, пишет Sky News. В городе
Сент-Луис повреждены более 5 тыс. домов.  Национальная метеорологическая служба США предупредила, что в ближайшее время шторм и торнадо направятся в сторону
штатов Техас, Канзас и Оклахома. Там ожидаются «сильные грозы, очень крупный град и разрушительные порывы ветра».


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--- ruGPT3medium summary ---
В результате разрушительного урагана "Харви" в американском штате Кентавр погибли по меньшей мере 37 человек.

--- ruGPT3small summary ---
По данным американской гражданской службы здравоохранения, от урагана в городе Сан-Франциско погибли по меньшей мере 273 человека, еще 291 человек находится в
больницах, сообщает Associated Press. Шторм "Мэтью" В пятницу "мэтья" обрушился на штат Кент. Он привел к повреждению линий электроп

--- ruT5 summary ---
По меньшей мере 27 человек погибли в результате торнадо и штормов в центральной части США.

--- mbart summary ---
Число погибших от торнадо в США увеличилось до 27 Как минимум 27 человек погибли в результате торнадо и штормов в центральной части США. Из них 25 находились в
штатах Миссури и Кентукки, 2 — в Северной Вирджинии. В штате Висконсин без света остались сотни тысяч человек.

--- mt5 summary ---
Число погибших от торнадо и штормов в США увеличилось до 27 человек.


In [29]:
text = input()
print('Новость')
print(textwrap.fill(text, width=WRAP_WIDTH))
summaries = summarize_text(text)
for model_name, summary in summaries.items():
    print(f"\n--- {model_name} summary ---")
    print(textwrap.fill(summary, width=WRAP_WIDTH))

Новость
Парусное судно ВМС Мекскики ARM Cuauhtemoc врезалось в Бруклинский мост в Нью-Йорке. Одна из его мачт в результате столкновений упала на палубу. Пострадали как
минимум 20 человек.  Cтолкновение произошло около 21:00 по Нью-Йорку (4:00 мск). По информации Bloomberg, на борту находились 277 человек.  Мэр Нью-Йорка Эрик
Адамс сообщил о 19 пострадавших. Четверо находятся в тяжелом состоянии. Людей продолжают доставать из воды.  Как писала New York Post, судно ARM Cuauhtemoc
прибыло в Нью-Йорк 13 мая. На нем находились в том числе 175 кадетов ВМС Мексики. Остановка стала третьей в восьмимесячном кругосветном путешествии. В
дальнейшем судно должно было зайти в порты в том числе Ямайки, Исландии и Португалии.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--- ruGPT3medium summary ---
Судно ВМС Мексики врезалась в здание суда и загорелось.

--- ruGPT3small summary ---
Корабли ВМС Колумбии и Новой Зеландии продолжают получать помощь и поддержку от властей Колумбийского региона. Президент Колумбия Хуан Мануэль Сантос поздравил
экипаж с возвращением домой. Он сказал, что на судне "остается большое количество людей, и они нуждаются в поддержке". Колумбанец получил ранение в шею во время
столкновения

--- ruT5 summary ---
Парусное судно ВМС Мексики врезалось в Бруклинский мост в Нью-Йорке.

--- mbart summary ---
В Нью-Йорке парусное судно ВМС Мекскики ARM Cuauhtemoc врезалось в Бруклинский мост в результате столкновений. Пострадали как минимум 20 человек. Мэр Нью-Йорка
Эрик Адамс сообщил о 19 пострадавших. Четверо находятся в тяжелом состоянии. Людей продолжают доставать из воды.

--- mt5 summary ---
По меньшей мере 20 человек погибли в результате столкновений между судном ВМС Мексики и мостом в Нью-Йорке.


In [32]:
text = input()
print('Новость')
print(textwrap.fill(text, width=WRAP_WIDTH))
summaries = summarize_text(text)
for model_name, summary in summaries.items():
    print(f"\n--- {model_name} summary ---")
    print(textwrap.fill(summary, width=WRAP_WIDTH))

Новость
Австрия выиграла «Евровидение» Австрийский исполнитель JJ выиграл конкурс «Евровидение-2025» с песней Wasted Love. Он набрал 436 баллов. Второе место заняла
Юваль Рафаэль из Израиля (357), третье —Томми Кэш из Эстонии (356). JJ победил в том числе в голосовании жюри, которое отдало певцу 258 баллов, говорится в
сообщении на сайте «Евровидения». Еще 176 баллов исполнитель получил по итогам голосования телезрителей  В десятку лидеров «Евровидения-2025» вошли:  Австрия
(436 баллов); Израиль (357); Эстония (356); Швеция (321); Италия (256); Греция (231); Франция (230); Албания (218); Украина (218); Швейцария (214). Меньше всех
баллов набрали Сан-Марино (27), Исландия (33), Испания (37). Всего в конкурсе участвовали 37 стран, в финале выступили 26 артистов.  «Евровидение-2025»
проходило в швейцарском Базеле с 13 по 17 мая. Россия последний раз участвовала в конкурсе в 2021 году.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--- ruGPT3medium summary ---
Австрия набрала 437 баллов;JJ стал победителем в трех возрастных категориях: 20-34, 35-39 и 40-44 года.

--- ruGPT3small summary ---
В финальном голосованием жюри выбор приняло решение признать Россию победителем и не допустить дальнейшей дискриминации в Европе по этническому, языковому и
религиозному признаку. Больше всего голосов набрала Черногория, набравшая 316 баллов в этом же конкурсе.Краткое сообщение: Автор словила австрию, которая стала

--- ruT5 summary ---
Австрийский певец JJ победил на песенном конкурсе «Евровидение-2025» в швейцарском Базеле.

--- mbart summary ---
JJ выиграл конкурс «Евровидение-2025» с песней Wasted Love. В десятку лидеров «Евровидения-2025» вошли Австрия, Израиль и Эстония.

--- mt5 summary ---
Победителем песенного конкурса "Евровидение-2025" стал австрийский исполнитель JJ.


In [19]:
data = pd.read_csv("final_data/test.csv")
article = data.loc[500, "text"]
sum = data.loc[500, "summary"]
print("Оригинал")
print(textwrap.fill(article, width=WRAP_WIDTH))
print("\nЭталонное summary:")
print(textwrap.fill(sum, width=WRAP_WIDTH))

Оригинал
Вашингтон осудил очередные пуски ракет Северной Кореей. "Запуски ракет КНДР угрожают самолетам гражданской авиации и морским судам, находящимся в этом регионе",
- говорится в заявлении внешнеполитического ведомства США. Ранее южнокорейское агентство Ренхап сообщило, что Северная Корея со своего восточного побережья
запустила три баллистические ракеты в сторону Японского моря. 6 января Северная Корея провела ядерное испытание, а 7 февраля запустила ракету-носитель с
искусственным спутником Земли, нарушив резолюцию Совета Безопасности ООН. В ответ в отношении КНДР были ужесточены международные санкции.

Эталонное summary:
Госдепартамент США выразил осуждение очередным запускам баллистических ракет Северной Кореей.


In [33]:
data = pd.read_csv("final_data/test.csv")
article = data.loc[20, "text"]
sum = data.loc[20, "summary"]
print("Оригинал")
print(textwrap.fill(article, width=WRAP_WIDTH))
print("\nЭталонное summary:")
print(textwrap.fill(sum, width=WRAP_WIDTH))

summaries = summarize_text(article)
for model_name, summary in summaries.items():
    print(f"\n--- {model_name} summary ---")
    print(textwrap.fill(summary, width=WRAP_WIDTH))

Оригинал
Число жертв теракта в метро Петербурга достигло 16 человек. Поезда ходили через станцию "Технологический институт" еще какое-то время после взрыва на соседних
путях "В пятницу в 06:05, находясь в крайне тяжелом состоянии, скончалась пострадавшая в теракте 1960 года рождения", - сообщила вице-губернатор города Анна
Митянина в Twitter. Таким образом, число жертв теракта сейчас составляет 16 человек, включая смертника. З апреля в Петербурге на перегоне между станциями
"Сенная площадь" и "Технологический институт" прогремел взрыв. Вскоре после этого на станции "Площадь Восстания" было обнаружено и обезврежено еще одно
самодельное взрывное устройство. Следственный комитет России возбудил уголовное дело по статье "теракт". По версии следствия, исполнителем теракта был гражданин
России Акбаржон Джалилов 1995 года рождения, уроженец Киргизии. Следователи полагают, что Акбаржон Джалилов сначала спрятал замаскированную бомбу на "Площади
Восстания", а затем подорвался в вагоне. В рамках 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--- ruGPT3medium summary ---
В Санкт Петербурге в результате взрыва поезда погибла 16-летняя девушка, еще двое пострадали. Девять человек получили ранения. Сейчас их состояние стабильно.

--- ruGPT3small summary ---
Количество жертв взрыва петербургского метро выросло до 16, погибли 12 человек и еще 18 были ранены, сообщили в пресс-службе петербургского ГУ МЧС. Число
пострадавших увеличилось до 20. Сейчас в больницы доставлены 12 раненых, 14 находятся в тяжелом критическом состоянии. Ранее в среду в центре Петербурга
прогремели два взрыва: четыре человека погибли и

--- ruT5 summary ---
Число жертв теракта в Санкт-Петербурге достигло 16 человек, в том числе смертник, сообщила вице-губернатор Санкт-Петербурга Анна Митянина.

--- mbart summary ---
Число жертв теракта в метро Петербурга достигло 16 человек, включая смертника. По предварительным данным, исполнителем теракта был гражданин России Акбаржон
Джалилов 1995 года рождения, уроженец Киргизии. В рамках расследования теракта были за

In [34]:
data = pd.read_csv("final_data/test.csv")
article = data.loc[200, "text"]
sum = data.loc[200, "summary"]
print("Оригинал")
print(textwrap.fill(article, width=WRAP_WIDTH))
print("\nЭталонное summary:")
print(textwrap.fill(sum, width=WRAP_WIDTH))

summaries = summarize_text(article)
for model_name, summary in summaries.items():
    print(f"\n--- {model_name} summary ---")
    print(textwrap.fill(summary, width=WRAP_WIDTH))

Оригинал
В Москве горел телецентр "Останкино". Возгорание произошло в цехе художественных декораций телецентра. Площадь возгорания составила 200 квадратных метров,
сообщили в Министерстве чрезвычайных ситуаций России. Из здания эвакуировали более тысячи работников телецентра. Из-за инцидента телеканал ОРТ перешел на
резервное вещание, а НТВ прервал комментарий к футбольному матчу. Пожар был потушен через несколько часов В МЧС назвали две основные версии пожара -
неисправность электропроводки и нарушение технологического процесса при окраске декораций.

Эталонное summary:
В московском телецентре "Останкино" вечером во вторник, 30 июля, загорелось одно из зданий, никто не пострадал.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--- ruGPT3medium summary ---
Пожар в телевизионном центре Останкино в Москве, начавшийся около шести часов утра, потушили через два часа.

--- ruGPT3small summary ---
На пульт "01" поступило сообщение о возгорании декорации к фильму "Идиот", в которой показано крушение российского самолета Су-24 на Синайском полуострове.
"Площадь пожара составила 150 квадратных м, площадь составляет 400 квадратных метра, обнаружены тела 45 погибших. Среди погибших - ведущий программы "Вести в
субботу

--- ruT5 summary ---
В московском телецентре "Останкино" загорелся цех художественных декораций.

--- mbart summary ---
В Москве горел телецентр «Останкино». Возгорание произошло в цехе художественных декораций. Площадь возгорания составила 200 квадратных метров. Из здания
эвакуировали более тысячи работников телецентра. Из-за инцидента телеканал ОРТ перешел на резервное вещание.

--- mt5 summary ---
В Москве горел телецентр "Останкино", а НТВ перешел на резервное вещание.


In [35]:
data = pd.read_csv("final_data/test.csv")
article = data.loc[500, "text"]
sum = data.loc[500, "summary"]
print("Оригинал")
print(textwrap.fill(article, width=WRAP_WIDTH))
print("\nЭталонное summary:")
print(textwrap.fill(sum, width=WRAP_WIDTH))

summaries = summarize_text(article)
for model_name, summary in summaries.items():
    print(f"\n--- {model_name} summary ---")
    print(textwrap.fill(summary, width=WRAP_WIDTH))

Оригинал
Вашингтон осудил очередные пуски ракет Северной Кореей. "Запуски ракет КНДР угрожают самолетам гражданской авиации и морским судам, находящимся в этом регионе",
- говорится в заявлении внешнеполитического ведомства США. Ранее южнокорейское агентство Ренхап сообщило, что Северная Корея со своего восточного побережья
запустила три баллистические ракеты в сторону Японского моря. 6 января Северная Корея провела ядерное испытание, а 7 февраля запустила ракету-носитель с
искусственным спутником Земли, нарушив резолюцию Совета Безопасности ООН. В ответ в отношении КНДР были ужесточены международные санкции.

Эталонное summary:
Госдепартамент США выразил осуждение очередным запускам баллистических ракет Северной Кореей.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--- ruGPT3medium summary ---
Вашингтон осуждает очередных пусков ракет со стороны Северной Кореи.

--- ruGPT3small summary ---
Президент США Барак Обама призвал Сеул и Вашингтон к продолжению диалога по ядерной проблеме, заявив, в частности, о необходимости сохранить "все имеющиеся
каналы поставки вооружений". Заявление Белого дома не сопровождается комментариями, заявил пресс-секретарь российского президента Дмитрий Песков. Пресс-
конференция прошла с участием руководителей Северной Кореи и Южной Кореи, которые

--- ruT5 summary ---
Соединенные Штаты осудили очередные пуски ракет Северной Кореей в сторону Японского моря.

--- mbart summary ---
Вашингтон осудил очередные пуски ракет Северной Кореей в сторону Японского моря, которые угрожают самолетам гражданской авиации и морским судам, находящимся в
этом регионе. Ранее южнокорейское агентство Ренхап сообщило, что Северная Корея со своего восточного побережья запустила три баллистические ракеты.

--- mt5 summary ---
США осудили очеред